In [12]:
# standard inputs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


sns.set(style='whitegrid')
#plt.style.use('seaborn-vibrant')
# Loading the raw dataset
df = pd.read_csv('../data/raw/vgsales.csv')

# First 5 rows
#print(df.head())

# Check info and structure
#print(df.info())

# See some summary stats
#print(df.describe(include='all'))

# Check for missing values
print(df.isnull().sum())

#df.describe(include='all')

Name                  2
Platform              0
Year_of_Release     269
Genre                 2
Publisher            54
NA_Sales              0
EU_Sales              0
JP_Sales              0
Other_Sales           0
Global_Sales          0
Critic_Score       8582
Critic_Count       8582
User_Score         6704
User_Count         9129
Developer          6623
Rating             6769
dtype: int64


In [ ]:
# Loading the raw dataset
df = pd.read_csv('../data/raw/vgsales.csv')

In [ ]:
print(df.info())

# Check for missing values
print(df.isnull().sum())